In [1]:
import pandas as pd

shop_data = pd.read_csv('data/02_shopping_tran.txt',
                       dtype={'RCT_NO':'str','PD_S_C':'str'})
shop_data.columns = shop_data.columns.str.lower()

shop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3641082 entries, 0 to 3641081
Data columns (total 9 columns):
id          int64
rct_no      object
biz_unit    object
pd_s_c      object
br_c        int64
de_dt       int64
de_hr       int64
buy_am      int64
buy_ct      int64
dtypes: int64(6), object(3)
memory usage: 250.0+ MB


In [2]:
shop_data.head()

,id,rct_no,biz_unit,pd_s_c,br_c,de_dt,de_hr,buy_am,buy_ct
0,4008,002108,A01,0215,2,20150216,13,59600,2
1,6379,002109,A01,0075,29,20150213,11,35000,1
2,6379,002109,A01,0149,4,20150115,10,85000,1
3,8002,002110,A01,0138,10,20151220,10,25000,1
4,8002,002110,A01,0138,10,20151220,10,21000,1


In [3]:
basket = shop_data.groupby(['rct_no','pd_s_c'])['buy_ct']\
.sum().unstack().reset_index().fillna(0).set_index('rct_no')
basket.head()

pd_s_c,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627
rct_no,,,,,,,,,,,,,,,,,,,,,
000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000005,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
print('basket shape = ',basket.shape)
print('# of unique rct_no = ',len(shop_data.rct_no.unique()))
print('# of unique pd_s_c = ',len(shop_data.pd_s_c.unique()))

basket shape =  (481778, 1627)
# of unique rct_no =  481778
# of unique pd_s_c =  1627


In [5]:
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

basket_sets = basket.applymap(encode_units)

In [6]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [7]:
#get frequent itemsets 
frequent_itemsets = apriori(basket_sets,min_support=0.005,use_colnames=True)

In [8]:
rules = association_rules(frequent_itemsets,metric='lift')
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0009),(0018),0.028299,0.039373,0.006351,0.224439,5.700339,0.005237,1.238622
1,(0018),(0009),0.039373,0.028299,0.006351,0.161316,5.700339,0.005237,1.158601
2,(0019),(0009),0.032691,0.028299,0.005932,0.181460,6.412175,0.005007,1.187115
3,(0009),(0019),0.028299,0.032691,0.005932,0.209623,6.412175,0.005007,1.223857
4,(0021),(0009),0.031126,0.028299,0.006740,0.216524,7.651217,0.005859,1.240244


In [9]:
frequent_itemsets

,support,itemsets
0,0.028474,[0001]
1,0.011954,[0002]
2,0.007371,[0003]
3,0.006588,[0004]
4,0.010129,[0005]
5,0.011630,[0006]
6,0.011345,[0007]
7,0.009091,[0008]
8,0.028299,[0009]
9,0.008693,[0010]


In [10]:
rules.tail()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
103,(1442),(1433),0.049355,0.035321,0.005955,0.120658,3.416010,0.004212,1.097046
104,(1444),(1442),0.034740,0.049355,0.006933,0.199558,4.043342,0.005218,1.187650
105,(1442),(1444),0.049355,0.034740,0.006933,0.140466,4.043342,0.005218,1.123004
106,(1620),(1442),0.053033,0.049355,0.006283,0.118474,2.400453,0.003666,1.078408
107,(1442),(1620),0.049355,0.053033,0.006283,0.127303,2.400453,0.003666,1.085104


In [11]:
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 9 columns):
antecedants           108 non-null object
consequents           108 non-null object
antecedent support    108 non-null float64
consequent support    108 non-null float64
support               108 non-null float64
confidence            108 non-null float64
lift                  108 non-null float64
leverage              108 non-null float64
conviction            108 non-null float64
dtypes: float64(7), object(2)
memory usage: 7.7+ KB


In [12]:
rules.to_csv('data/rules.csv')